In [47]:
filepath = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/"
filepath

'/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/'

In [48]:
import pretty_midi
import os

def extract_guitar_and_drums(midi_file):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""    
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))
        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))
    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]
    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    
    return song_dict


In [49]:
import os 

directory = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/"

new_directory = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/data/"

midi_files = []

for filename in os.listdir(directory):
    full_path = os.path.join(directory, filename)
    midi_files.append(full_path)

In [50]:
print(midi_files)

['/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_The_Things_We_Do_for_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Im_Not_In_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/ABC_-_The_Look_of_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/1910_Fruitgum_Company_-_Simon_Says.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/Aaron_Neville_-_Tell_It_Like_It_Is.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/911_-_The_Day_We_Find_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/38_Special_-_Fantasy_Girl.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/ABC_-_Poison_Arrow.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/38_Special_-_Caught_Up_In_You.mid']


In [51]:

list_ = []

for file in midi_files:
    dict_ = extract_guitar_and_drums(full_path)
    list_.append(dict_)


In [52]:
for element in list_:
    print(element)

{'title': '38_Special_-_Caught_Up_In_You', 'down_beats': array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178., 180., 182., 184., 186.]), 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': '38_Special_-_Caught_Up_In_You', 'down_beats': array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.

In [53]:
len(list_)

10

In [54]:
list_[-1]

{'title': '38_Special_-_Caught_Up_In_You',
 'down_beats': array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
         22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
         44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
         66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
         88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
        110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
        132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
        154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
        176., 178., 180., 182., 184., 186.]),
 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"),
 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}

In [55]:
list_of_string = []

for element in list_:
    string = ""
    for note in element["guitar"].notes:
        delta = "delta=" + str(note.end - note.start) + " "
        #start_time = "start_time=" + str(int(note.start)) + " "
        #end_time = "end_time=" + str(int(note.end)) + " "
        pitch = "pitch=" + str(note.pitch) + " "
        velocity = "velocity=" + str(note.velocity) + " "
        string = string + delta + pitch + velocity
    list_of_string.append(string)


In [56]:
len(list_of_string)

10

In [57]:
note = dict_["guitar"].notes[0].start
type(note)

numpy.float64

In [58]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

from transformers import PreTrainedTokenizerFast


tokenizer1 = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer1.pre_tokenizer = WhitespaceSplit()
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

string = list_of_string[0]
tokenizer1.train_from_iterator(string, trainer=trainer)
tokenizer1.save("tokenizer.json")

tokenizer1 = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")
tokenizer1.add_special_tokens({'pad_token': '[PAD]'})

0

In [59]:
len(tokenizer1.get_vocab())

29

In [60]:
tokenizer1.get_vocab()

{'1': 13,
 'e': 9,
 '[MASK]': 4,
 'i': 10,
 'a': 17,
 '2': 26,
 'v': 22,
 'p': 21,
 '=': 6,
 'c': 8,
 '3': 27,
 'l': 11,
 '6': 5,
 '7': 12,
 'd': 18,
 '[UNK]': 0,
 't': 7,
 '.': 16,
 'y': 23,
 '5': 25,
 '[SEP]': 2,
 '9': 14,
 '[PAD]': 3,
 '8': 28,
 '[CLS]': 1,
 'o': 20,
 'h': 19,
 '4': 24,
 '0': 15}

In [61]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

from transformers import PreTrainedTokenizerFast


tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = WhitespaceSplit()
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

def get_training_corpus():
    for element in list_of_string:
        yield element
    
training_corpus = get_training_corpus()
tokenizer.train_from_iterator(training_corpus, trainer=trainer)
tokenizer.save("tokenizer.json")

tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")

In [62]:
len(tokenizer.get_vocab())

33

In [63]:
tokenizer.get_vocab()

{'pitch=42': 27,
 'delta=0.1666666666666643': 8,
 'delta=0.5833333333333428': 29,
 '[PAD]': 3,
 'delta=0.65625': 26,
 '[SEP]': 2,
 'pitch=57': 12,
 'pitch=56': 15,
 'delta=0.16666666666666785': 13,
 'pitch=54': 16,
 'delta=0.16666666666666607': 19,
 'delta=0.5': 22,
 'pitch=69': 28,
 '[CLS]': 1,
 'pitch=59': 10,
 'pitch=61': 11,
 'pitch=68': 17,
 'velocity=79': 5,
 'pitch=47': 18,
 'pitch=71': 20,
 'delta=0.3333333333333428': 32,
 'pitch=66': 14,
 'delta=0.1666666666666714': 6,
 '[UNK]': 0,
 'delta=0.5729166666666572': 30,
 '[MASK]': 4,
 'delta=0.1666666666666572': 7,
 'pitch=58': 21,
 'pitch=51': 31,
 'pitch=52': 23,
 'pitch=64': 9,
 'pitch=63': 24,
 'pitch=49': 25}

In [64]:
string

'delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.1666

## Check the range of values that we have

In [65]:
delta = []
pitch = []
velocity = []

for element in list_:
    for note in element["guitar"].notes:
        delta.append(note.end - note.start)
        pitch.append(note.pitch)
        velocity.append(note.velocity)


delta = set(delta)
pitch = set(pitch)
velocity = set(velocity)

In [66]:
delta

{0.1666666666666572,
 0.1666666666666643,
 0.16666666666666607,
 0.16666666666666785,
 0.1666666666666714,
 0.3333333333333428,
 0.5,
 0.5729166666666572,
 0.5833333333333428,
 0.65625}

In [67]:
pitch

{42, 47, 49, 51, 52, 54, 56, 57, 58, 59, 61, 63, 64, 66, 68, 69, 71}

In [68]:
velocity

{79}

In [69]:
delta = []
pitch = []
velocity = []
name = ""
for note in list_[1]["guitar"].notes:
    delta.append(note.end - note.start)
    pitch.append(note.pitch)
    velocity.append(note.velocity)

name = list_[0]["title"]
delta = set(delta)
pitch = set(pitch)
velocity = set(velocity)

In [70]:
name

'38_Special_-_Caught_Up_In_You'

In [71]:
delta

{0.1666666666666572,
 0.1666666666666643,
 0.16666666666666607,
 0.16666666666666785,
 0.1666666666666714,
 0.3333333333333428,
 0.5,
 0.5729166666666572,
 0.5833333333333428,
 0.65625}

In [72]:
pitch

{42, 47, 49, 51, 52, 54, 56, 57, 58, 59, 61, 63, 64, 66, 68, 69, 71}

In [73]:
velocity

{79}

In [74]:
list_[7]["guitar"].notes[-1]

Note(start=187.750000, end=187.916667, pitch=47, velocity=79)

In [75]:
list_[5]["guitar"].notes[-1]

Note(start=187.750000, end=187.916667, pitch=47, velocity=79)

In [76]:
for e in dir(tokenizer):
    print(e)

SPECIAL_TOKENS_ATTRIBUTES
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__len__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_add_tokens
_additional_special_tokens
_auto_class
_batch_encode_plus
_bos_token
_call_one
_cls_token
_compile_jinja_template
_convert_encoding
_convert_id_to_token
_convert_token_to_id_with_added_voc
_create_repo
_decode
_decode_use_source_tokenizer
_encode_plus
_eos_token
_eventual_warn_about_too_long_sequence
_eventually_correct_t5_max_length
_from_pretrained
_get_files_timestamps
_get_padding_truncation_strategies
_in_target_context_manager
_mask_token
_pad
_pad_token
_pad_token_type_id
_processor_class
_save_pretrained
_sep_token
_set_processor_class
_switch_to_input_mode
_switch_to_target_mode
_tokenizer
_unk_token
_upload_modified_files
add_special_tokens
add_

In [77]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [78]:
tokenizer.model_input_names[0]

'input_ids'

## Trying REMI tokenization

In [82]:
#pip install miditok

In [83]:
#pip install torch

In [85]:
def tracks_to_bars(song_dict: dict):
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        try:
            end_time = down_beats_array[index+1]            
            guitar_bar = []
            drums_bar = []
            for guitar_note, drum_note in zip(guitar.notes, drums.notes):
                if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                    guitar_bar.append(guitar_note)    
                if (drum_note.start >= start_time) and (drum_note.end < end_time):
                    drums_bar.append(drum_note)
            drums_bars_list.append(drums_bar)
            guitar_bars_list.append(guitar_bar)
        except:
            print("done")
            pass
    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict
    

In [88]:
type(list_[0])

dict

In [90]:
new_list = []

for e in list_:
    new_list.append(tracks_to_bars(e))

done
done
done
done
done
done
done
done
done
done


{'song_title': '38_Special_-_Caught_Up_In_You',
 'guitar_bars': [[],
  [],
  [],
  [],
  [],
  [Note(start=10.000000, end=10.166667, pitch=71, velocity=79),
   Note(start=10.000000, end=10.166667, pitch=64, velocity=79),
   Note(start=10.250000, end=10.416667, pitch=71, velocity=79),
   Note(start=10.250000, end=10.416667, pitch=64, velocity=79),
   Note(start=10.500000, end=10.666667, pitch=71, velocity=79),
   Note(start=10.500000, end=10.666667, pitch=64, velocity=79),
   Note(start=10.750000, end=10.916667, pitch=71, velocity=79),
   Note(start=10.750000, end=10.916667, pitch=63, velocity=79),
   Note(start=11.000000, end=11.166667, pitch=71, velocity=79),
   Note(start=11.000000, end=11.166667, pitch=63, velocity=79),
   Note(start=11.250000, end=11.416667, pitch=71, velocity=79),
   Note(start=11.250000, end=11.416667, pitch=63, velocity=79),
   Note(start=11.500000, end=11.666667, pitch=71, velocity=79),
   Note(start=11.500000, end=11.666667, pitch=63, velocity=79),
   Note(sta

In [95]:
ratoli = new_list[0]["guitar_bars"]

In [96]:
len(ratoli)

93

In [133]:
type(new_list[0]["guitar_bars"][15][1])

pretty_midi.containers.Note

In [140]:
from miditok import REMI, TokenizerConfig
from symusic import Score

# Crear una configuración para el tokenizador
tokenizer = REMI()

notes_ = []
for note in new_list[0]["guitar_bars"][15]:
    notes_.append(note)



tokens = tokenizer(filename)

tokens


[TokSequence(tokens=['Bar_None', 'Bar_None', 'Bar_None', 'Position_0', 'Pitch_33', 'Velocity_99', 'Duration_2.4.8', 'Position_21', 'Pitch_33', 'Velocity_99', 'Duration_0.3.8', 'Position_24', 'Pitch_45', 'Velocity_99', 'Duration_0.2.8', 'Position_27', 'Pitch_47', 'Velocity_99', 'Duration_0.3.8', 'Position_29', 'Pitch_45', 'Velocity_99', 'Duration_0.2.8', 'Bar_None', 'Position_0', 'Pitch_33', 'Velocity_99', 'Duration_2.4.8', 'Position_21', 'Pitch_33', 'Velocity_99', 'Duration_1.1.8', 'Bar_None', 'Position_0', 'Pitch_33', 'Velocity_99', 'Duration_2.4.8', 'Position_21', 'Pitch_33', 'Velocity_99', 'Duration_0.3.8', 'Position_24', 'Pitch_45', 'Velocity_99', 'Duration_0.2.8', 'Position_27', 'Pitch_47', 'Velocity_99', 'Duration_0.3.8', 'Position_29', 'Pitch_45', 'Velocity_99', 'Duration_0.3.8', 'Bar_None', 'Position_0', 'Pitch_33', 'Velocity_99', 'Duration_2.3.8', 'Position_21', 'Pitch_33', 'Velocity_99', 'Duration_1.1.8', 'Bar_None', 'Position_0', 'Pitch_33', 'Velocity_99', 'Duration_2.3.8', 

In [124]:
for e in dir(tokenizer):
    print(e)

_MIDITokenizer__convert_resampling_ratios_ticks_to_idx
_MIDITokenizer__create_durations_tuples
_MIDITokenizer__create_pitch_bends
_MIDITokenizer__create_rests
_MIDITokenizer__create_tempos
_MIDITokenizer__create_time_signatures
_MIDITokenizer__create_tpb_per_ts
_MIDITokenizer__create_tpb_ticks_to_tokens
_MIDITokenizer__create_tpb_to_ticks_array
_MIDITokenizer__create_tpb_tokens_to_ticks
_MIDITokenizer__create_vocabulary
_MIDITokenizer__get_from_voc
_MIDITokenizer__vocab_base_inv
__abstractmethods__
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getitem__
__gt__
__hash__
__init__
__init_subclass__
__le__
__len__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__slots__
__str__
__subclasshook__
__weakref__
_abc_impl
_add_additional_tokens_to_vocab_list
_add_note_tokens_to_vocab_list
_add_special_tokens_to_types_graph
_add_time_events
_adjust_durations
_adjust_offset_spanning_acr

In [125]:
tokenizer.__call__

<bound method MIDITokenizer.__call__ of 282 tokens with ('I', 'T') io format, without BPE>

In [115]:
# Función para convertir una nota a un token adecuado para el tokenizador
def note_to_token(note):
    # Aquí debes implementar la lógica para convertir una nota a un token
    # Puede ser algo como "Pitch_Velocity_Duration" dependiendo de cómo quieras representar la nota
    # Asegúrate de que el formato del token sea compatible con el vocabulario del tokenizador
    # Por ejemplo:
    return f"Pitch_{note.pitch}_Velocity_{note.velocity}_Duration_{note.duration}"


In [116]:
# Tokenización de los bares
tokenized_bars = []
for bar in ratoli:
    # Verificar si el bar tiene al menos una nota antes de tokenizarlo
    if bar:
        # Convertir las notas a una secuencia adecuada para el tokenizador
        note_sequence = [note_to_token(note) for note in bar]
        tokens = tokenizer(note_sequence)  # Tokenizar el bar utilizando REMI
        tokenized_bars.append(tokens)
    else:
        # Agregar una secuencia de tokens vacía para los bares sin notas
        tokenized_bars.append([])



AttributeError: 'list' object has no attribute 'tokens'

In [118]:
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score

filename = '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_The_Things_We_Do_for_Love.mid'

# Creating a multitrack tokenizer configuration, read the doc to explore other parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)

# Loads a midi, converts to tokens, and back to a MIDI
midi = Score(filename)
tokens = tokenizer(midi)  # calling the tokenizer will automatically detect MIDIs, paths and tokens
converted_back_midi = tokenizer(tokens)  # PyTorch / Tensorflow / Numpy tensors supported

# Trains the tokenizer with BPE, and save it to load it back later
midi_paths = list(Path("path", "to", "midis").glob("**/*.mid"))
tokenizer.learn_bpe(vocab_size=30000, files_paths=midi_paths)
tokenizer.save_params(Path("path", "to", "save", "tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Creates a Dataset and a collator to be used with a PyTorch DataLoader to train a model
dataset = DatasetTok(
    files_paths=midi_paths,
    min_seq_len=100,
    max_seq_len=1024,
    tokenizer=tokenizer,
)
collator = DataCollator(
    tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"]
)
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset=dataset, collate_fn=collator)
for batch in data_loader:
    print("Train your model on this batch...")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-65e9b173-1816ea5964279e7711ebfe4f;0b9c5df1-6cb3-4f22-8209-0a3da7459b34)

Invalid username or password.

Afternoon trying

In [157]:
bars = []
for song in new_list:
    for bar in song["guitar_bars"]:
        bars.append(str(bar))
    if song == new_list[0]:
        print(len(bars))

93


In [156]:
len(bars)

930

In [153]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

#from transformers import PreTrainedTokenizerFast


tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
#tokenizer.pre_tokenizer = WhitespaceSplit()

training_data = bars

trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

    
#training_corpus = get_training_corpus()
tokenizer.train_from_iterator(training_data, trainer=trainer)


In [154]:
tokenizer.get_vocab()

{'[Note(start=46.000000, end=46.166667, pitch=61, velocity=79), Note(start=46.000000, end=46.166667, pitch=54, velocity=79), Note(start=46.250000, end=46.416667, pitch=61, velocity=79), Note(start=46.250000, end=46.416667, pitch=54, velocity=79), Note(start=46.500000, end=46.666667, pitch=61, velocity=79), Note(start=46.500000, end=46.666667, pitch=54, velocity=79), Note(start=46.750000, end=46.916667, pitch=61, velocity=79), Note(start=46.750000, end=46.916667, pitch=54, velocity=79), Note(start=47.000000, end=47.166667, pitch=61, velocity=79), Note(start=47.000000, end=47.166667, pitch=54, velocity=79), Note(start=47.250000, end=47.416667, pitch=61, velocity=79), Note(start=47.250000, end=47.416667, pitch=54, velocity=79), Note(start=47.500000, end=47.666667, pitch=61, velocity=79), Note(start=47.500000, end=47.666667, pitch=54, velocity=79), Note(start=47.750000, end=47.916667, pitch=68, velocity=79), Note(start=47.750000, end=47.916667, pitch=61, velocity=79)]': 66,
 '[Note(start=1

In [155]:
len(tokenizer.get_vocab())

93

In [ ]:
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
) 

def get_training_corpus():
    for element in list_of_string:
        yield element